# Natural Language Processing + Decision Science

👩🏻‍🏫 In this challenge, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products and the sellers on Olist

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import unidecode as unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?
    - the wort sellers ?

## (0) Setup 🔨

First we'll load up the DataFrame which contains all the relevant information on Olist reviews!

In [ ]:
df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/olist/olist_reviews.csv")

In [ ]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review
0,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
1,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela ...
2,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo aparelho eficiente. no site a marca ...
3,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendo Vendedor confiável, produto ok..."


## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews. **Beware NLTK has no Portuguese lemmatizer** (often but they do have a stemmer `nltk.stem.RSLPStemmer`. 

In [ ]:
from nltk.stem import RSLPStemmer
nltk.download("rslp")
stemmer = RSLPStemmer()

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words('portuguese')) ## defining stopwords

def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.lower() ## lowercasing
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## removing numbers
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## removing punctuation
    # Advanced cleaning
    tokenized_sentence = word_tokenize(sentence) ## tokenizing

    tokenized_sentence = [w for w in tokenized_sentence
                                  if not w in stop_words] ## remove stopwords
    stemmed_sentence = [stemmer.stem(word)
              for word in tokenized_sentence] ## get word stems
    decoded_sentence = [unidecode.unidecode(w) for w in stemmed_sentence] ## remove accents

    cleaned_sentence = ' '.join(decoded_sentence) ## join back into a string

    return cleaned_sentence

In [ ]:
df["full_review_cleaned"] = df["full_review"].apply(cleaning)
df["full_review_cleaned"]

3                               receb bem ant praz estipul
4        parabem loj lannist ador compr internet segur ...
9        recom aparelh efici sit marc aparelh impress d...
12                             pouc travandopel val ta boa
15          sup recom vend confi produt ok entreg ant praz
                               ...                        
98639                      produt receb acord compr realiz
98642    entreg dentr praz produt cheg cond perfeit sat...
98648    produt envi nf exist vend nf cert fic aguard e...
98654          excel mochil entreg sup rapid sup recom loj
98656          produt cheg ja devolv poi defeit segur carg
Name: full_review_cleaned, Length: 36148, dtype: object

## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

😱 What is the proportion of reviews with a rating between 1 and 3 ? 

In [ ]:
round(df["review_score"].value_counts(normalize = True),2)

5    0.57
4    0.16
1    0.13
3    0.09
2    0.05
Name: review_score, dtype: float64

🕵🏻‍♂️ Let's focus on these reviews...

In [ ]:
threshold = 3
bad = df["review_score"] <= threshold
df = df[bad]
df.shape

(9658, 7)

In [ ]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned
53,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,receb soment control mide split estil falt con...
75,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,compr dua unidad receb agor fac
77,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produt bom porem vei mim condiz fot anunci
78,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produt inferi mal acab
89,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entreg praz


### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this challenge.

In [ ]:
vectorizer = TfidfVectorizer(max_df = 0.75, max_features = 5000, ngram_range=(1,2))
vectorized_reviews = pd.DataFrame(vectorizer.fit_transform(df["full_review_cleaned"]).toarray(),
                                 columns = vectorizer.get_feature_names_out())

print(f" vectorized_reviews.shape = {vectorized_reviews.shape}")

 vectorized_reviews.shape = (9658, 5000)


### (3.3) LDA

🕵🏻‍♂️ Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [ ]:
n_components = 3

lda = LatentDirichletAllocation(n_components = n_components)
lda.fit(vectorized_reviews)

LatentDirichletAllocation(n_components=3)

#### Document Mixture (of Topics)

In [ ]:
document_mixture = lda.transform(vectorized_reviews)
document_mixture.shape

(9658, 3)

In [ ]:
round(pd.DataFrame(document_mixture,
                   columns = [f"Topic {i+1}" for i in range(n_components)])
      ,2)

,Topic 1,Topic 2,Topic 3
0,0.09,0.72,0.19
1,0.82,0.08,0.10
2,0.38,0.54,0.08
3,0.38,0.52,0.10
4,0.13,0.74,0.13
...,...,...,...
9653,0.64,0.08,0.28
9654,0.78,0.11,0.11
9655,0.12,0.07,0.81
9656,0.08,0.21,0.71


👉 Let's report the most important topic for each review

In [ ]:
df["most_important_topic"] = np.argmax(document_mixture, axis = 1)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic
53,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,receb soment control mide split estil falt con...,1
75,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,compr dua unidad receb agor fac,0
77,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produt bom porem vei mim condiz fot anunci,1
78,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produt inferi mal acab,1
89,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entreg praz,1


#### Topic Mixture (of Words)

In [ ]:
topic_mixture = pd.DataFrame(lda.components_,
                             columns = vectorizer.get_feature_names_out())
topic_mixture.shape

(3, 5000)

In [ ]:
topic_mixture

abaix      abert   aborrec        abr  abr caix  abr cham  abr embal  \
0  0.345876   0.797842  0.882652  14.645659  1.856260  1.000331   0.966103   
1  3.358988  10.342535  1.893466   0.392630  0.374247  0.344501   0.407026   
2  1.302236   2.846713  0.633842   9.432817  0.970373  1.208655   3.016995   

   abr fech  abr reclam     abril    absurd  absurd ser       acab  \
0  1.770665    0.956460  1.818548  3.611221    0.333866  11.995042   
1  0.349245    0.336537  0.378245  2.389165    0.333990  22.466129   
2  0.335818    5.366374  0.358778  7.349801    1.055325   0.424713   

   acab compr  acab deix  acab fic  acab produt  acab receb  acab ruim  \
0    0.413884   2.653395  0.346763     0.697713    1.073346   3.948846   
1    1.646717   0.399859  1.933416     1.547789    1.623781   0.352547   
2    0.360411   0.335481  0.335627     0.342626    0.432903   0.334499   

      aceit     acend     acert     acess  acessori        ach  ach absurd  \
0  0.539709  2.669316  1.865408  1.479269  0.411155  19.720096    0.342140   
1  1.421926  0.358210  0.339912  1.165725  1.317951  46.716740    0.639456   
2  5.862351  0.336654  0.465151  1.017120  3.232977   6.591181    1.131487   

    ach bem   ach car  ach cord  ach demor   ach dev  ach entreg  ach estranh  \
0  0.365180  0.369158  0.333725   0.339336  0.635963    0.897428     0.342598   
1  2.170997  1.553939  1.128972   5.351313  2.453315    1.179014     0.478415   
2  0.338148  0.389843  0.333689   0.338796  2.502949    0.995533     1.678699   

   ach fornec  ach fret   ach iri  ach mant  ach mater  ach melhor  ach part  \
0    1.226524  0.338754  1.206936  0.335626   0.952414    1.943247  1.310425   
1    0.334279  2.617970  0.410160  0.353218   2.466815    0.483763  0.410160   
2    0.348937  0.340055  0.344715  2.104719   0.372348    0.340003  0.337037   

   ach pequen  ach pess  ach pouc  ach praz  ach prec  ach produt  ach qual  \
0    3.527880  0.343582  0.345103  0.346772  0.334265    8.353182  1.937034   
1    1.604570  1.867788  4.625597  3.237031  2.611108    1.619019  0.649979   
2    0.336128  0.503872  0.334752  0.341067  0.334882    0.357440  0.339327   

    ach vei      acid      acim     acion       aco  acompanh  acompanh ped  \
0  1.140762  1.823822  1.706708  1.880687  1.897357  1.187758      0.362436   
1  1.414576  0.339555  0.551848  0.715629  0.378911  1.365948      0.342524   
2  0.362416  0.338372  0.700659  0.995404  0.770448  7.232404      2.343945   

   acompanh produt  acondicion    acontec  acontec compr  acontec produt  \
0         0.390394    0.363171  11.774365       0.355412        2.936927   
1         0.341868    2.785569   0.358903       0.769155        0.334684   
2         2.196671    0.337021   7.875404       1.072819        0.336343   

      acord  acord descr   acostum  acostum compr   acredit  acredit dev  \
0  7.822665     1.450069  0.357155       0.334743  8.172533     1.368762   
1  2.693746     0.337342  2.365484       1.602548  3.921273     0.337677   
2  1.057779     0.475042  0.346554       0.353643  0.367298     0.364806   

   acredit produt  acredit sid     acril  acril fic      acus     adapt  \
0        1.469454     1.383228  0.373351   0.333949  0.667874  3.267601   
1        0.335659     0.387870  4.868589   1.906148  0.345270  0.367095   
2        0.367487     0.337062  0.344803   0.333895  2.678625  2.285772   

      adequ      ader      ades  ades mal  ades pared  ades pra    adiant  \
0  0.357116  0.349439  6.699214  0.988792    1.193296  1.108973  0.957583   
1  1.093848  2.190791  2.587814  0.638277    0.352212  0.333981  0.555710   
2  4.837309  0.351114  1.563518  0.348165    1.079997  0.333960  1.686714   

        ador  ador produt    adquir  adquir consegu  adquir produt     adult  \
0   0.342615     0.336333  5.445166        0.367705       0.491771  0.538832   
1  10.273086     1.757073  2.081127        1.647577       0.419607  0.335262   
2   0.337228     0.339958  8.400905        0.335719       3.11

#### Topics

🎁 We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [ ]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [ ]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))


🕵🏻‍♂️ Print the topics with their topwords:

In [ ]:
print_topics(vectorizer, lda, topwords = 5)

--------------------
Topic 0:
[('produt', 262.13), ('receb', 185.32), ('bom', 183.61), ('receb produt', 121.02), ('compr', 109.82)]
--------------------
Topic 1:
[('produt', 142.13), ('praz', 128.77), ('gost', 112.14), ('vei', 104.25), ('entreg', 87.32)]
--------------------
Topic 2:
[('entreg', 179.57), ('compr', 157.37), ('receb', 142.94), ('produt', 117.38), ('ped', 112.4)]


🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _produto = product_
- _recomendo = recommend (não recomendo == not recommend)_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _errado = wrong_
- _gostaria = I would like to..._
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [ ]:
topic_word_mixture = [topic_word(vectorizer, lda, topic, topwords = 5, with_weights = False)
                      for topic in lda.components_]
topic_word_mixture

[['produt', 'receb', 'bom', 'receb produt', 'compr'],
 ['produt', 'praz', 'gost', 'vei', 'entreg'],
 ['entreg', 'compr', 'receb', 'produt', 'ped']]

In [ ]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

In [ ]:
df[["review_id",
        "review_score",
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head()

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
53,e233e51d11511bf30e568c76360ace52,1,eletronicos,receb soment control mide split estil falt con...,1,"[produt, praz, gost, vei, entreg]"
75,8b230a1373c6dc4bd867099fda1d7039,3,ferramentas_jardim,compr dua unidad receb agor fac,0,"[produt, receb, bom, receb produt, compr]"
77,cb2fc3e5711b5ae85e0491ee18af63ed,3,beleza_saude,produt bom porem vei mim condiz fot anunci,1,"[produt, praz, gost, vei, entreg]"
78,60c714ed14cef913944a3147094a4742,1,moveis_decoracao,produt inferi mal acab,1,"[produt, praz, gost, vei, entreg]"
89,0bd4dcc4f6c4621baf37f73495cad8c4,3,esporte_lazer,entreg praz,1,"[produt, praz, gost, vei, entreg]"


## (3.4) Pipeline Tf-Idf and LDA

In [ ]:
from sklearn import set_config
set_config("diagram")

🔨 Create a Pipeline that chains your previous Vectorizer and the LDA.

Fit it on the cleaned texts.

In [ ]:
%%time
# Parameters
max_df = 0.75
max_features = 5000
ngram_range = (1,2)

# Pipeline Vectorizer + LDA
pipeline = make_pipeline(
    TfidfVectorizer(max_df = max_df,
                    max_features = max_features,
                    ngram_range = ngram_range),
    LatentDirichletAllocation(n_components = n_components)
)

# Fit the pipeline on the cleaned texts
pipeline.fit(df["full_review_cleaned"])

CPU times: user 16.9 s, sys: 104 ms, total: 17 s
Wall time: 17.2 s


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.75, max_features=5000,
                                 ngram_range=(1, 2))),
                ('latentdirichletallocation',
                 LatentDirichletAllocation(n_components=3))])

💡 If you try to access the components with `pipeline.components_`, it will NOT work because Pipeline doesn't have a `components_`. However, you can use `pipeline._final_estimator` to access the LDA. And from this, you can access the topics!

In [ ]:
pipeline._final_estimator

LatentDirichletAllocation(n_components=3)

In [ ]:
pipeline._final_estimator.components_

array([[ 0.41597695,  1.07563854,  0.79144829, ...,  0.34021647,
         1.07409767,  4.95398374],
       [ 4.19378792, 12.55826656,  2.27786812, ...,  0.43961164,
         7.55108322,  1.70498218],
       [ 0.39733384,  0.35318553,  0.34064329, ...,  2.73317086,
         0.59691253,  0.33789851]])

**Document Mixture** with the Pipeline:

In [ ]:
# Transform the original cleaned texts with the pipeline
# Indeed, there is no need to get the vectorized texts first since it's done through the Pipeline
document_mixture = pipeline.transform(df["full_review_cleaned"])
document_mixture.shape

(9658, 3)

**Topic Mixture** with the Pipeline:

In [ ]:
topic_mixture = pd.DataFrame(pipeline._final_estimator.components_)
topic_mixture.shape

(3, 5000)

In [ ]:
topic_mixture

0          1         2          3         4         5         6     \
0  0.415977   1.075639  0.791448  11.394363  0.577176  0.739669  2.891858   
1  4.193788  12.558267  2.277868   7.358201  1.514634  1.350671  0.631641   
2  0.397334   0.353186  0.340643   5.718543  1.109070  0.463148  0.866624   

       7         8         9         10        11         12        13    \
0  1.777981  0.350594  0.361952  6.551308  0.337633  25.743897  1.498567   
1  0.334793  2.308211  0.886418  3.599195  0.333849   8.784569  0.367148   
2  0.342954  4.000566  1.307201  3.199684  1.051700   0.357417  0.555297   

       14        15        16        17        18        19        20    \
0  0.385617  1.918888  1.087159  1.006283  3.934876  1.640698  2.259373   
1  2.667997  0.356840  1.160426  0.819682  0.366205  5.706427  0.767193   
2  0.335120  0.340078  0.340543  1.304065  0.334810  0.476860  0.337614   

       21        22        23         24        25        26        27    \
0  0.353267  1.291931  0.376385  48.463867  1.438465  1.507899  1.619909   
1  0.340306  1.963142  3.140884  23.732829  0.336438  1.031142  0.357605   
2  1.976899  0.407041  1.444814   0.831321  0.338180  0.335284  0.335426   

       28        29        30        31        32        33        34    \
0  0.338529  5.345282  1.608456  1.488179  0.338124  1.234540  2.608047   
1  0.333895  0.349134  2.606109  1.248035  1.823221  0.333977  0.339949   
2  1.123962  0.335030  1.377662  0.335760  0.338366  0.341223  0.348784   

       35        36        37        38        39        40        41    \
0  0.585912  0.341100  0.380642  0.362593  1.261059  4.765850  1.932320   
1  1.023275  0.342535  3.068439  2.068322  0.460671  0.367481  0.435781   
2  0.352623  2.109928  0.342497  0.336097  0.335892  0.335248  0.347141   

       42        43        44        45        46        47        48    \
0  3.237738  3.237806  0.385163  5.521578  2.252794  1.928555  1.824474   
1  1.732205  0.345871  2.559501  4.467412  0.338958  0.596266  0.342852   
2  0.335509  0.341194  0.335591  0.340651  0.334587  0.392932  0.334422   

       49        50        51        52        53        54        55    \
0  1.280066  1.346114  0.455045  4.442469  0.351304  2.067582  2.768229   
1  1.277910  1.529637  0.433941  2.261800  1.442918  0.503699  0.372760   
2  0.401239  0.715970  2.157731  3.081841  1.254682  0.357652  0.344773   

        56        57        58        59        60        61        62    \
0   1.675004  0.344913  0.337517  3.377228  0.334721  0.855807  0.517962   
1   7.157096  1.403372  0.360750  7.343424  1.585344  1.876578  1.433594   
2  11.176572  0.449101  2.909688  0.853538  0.342387  0.336809  0.339378   

       63        64        65        66        67        68        69    \
0  7.283536  1.288174  1.416490  0.365376  2.158838  0.344088  0.336008   
1  4.757550  0.362021  0.335004  1.406040  3.089142  1.895413  0.366576   
2  0.420018  0.421051  0.421106  0.336744  0.338763  0.334491  2.989184   

       70        71        72        73        74        75        76    \
0  5.175135  3.480636  0.595594  1.535935  0.399618  0.339440  0.333954   
1  0.358265  2.441090  0.341251  8.937432  1.239600  1.896101  1.109007   
2  0.387070  0.366547  1.954499  0.377179  0.336017  0.389964  0.333952   

       77         78        79         80        81        82        83    \
0  1.148346  10.227661  1.763255   0.470677  0.334039  0.343062  0.372925   
1  1.024861   0.385826  0.334681  11.070230  1.682753  3.235611  1.234690   
2  1.026801   0.339442  0.335428   4.386291  0.334209  0.443609  0.911413   

       84        85        86        87        88        89        90    \
0  0.366030  0.333954  0.351924  0.373150  8.538113  6.104628  4.077935   
1  2.331357  1.109007  1.121700  0.618804  0.340142  0.338843  0.382968   
2  0.334961  0.333952  1.002649  1.494857  0.341807  0.343262  1.374347   

       91         92        93        94        95        96      

## (4) 🎁 Product Categories

### (4.1) Groupby product categories

📈 Group the dataset by `product_category_name` and inspect their performances

In [ ]:
# Product categories by performance - look at the count, mean, median and std
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Removing products which were sold less than a certain times for the analysis
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Sorting the product categories by performance
product_categories = product_categories.sort_values(by = [('review_score', 'mean'),
                                                          ('review_score', 'std')],
                                                    ascending = [False, True])
product_categories

review_score                           
                                         count      mean median       std
product_category_name                                                    
consoles_games                              92  1.956522    2.0  0.924787
fashion_bolsas_e_acessorios                155  1.948387    2.0  0.917319
malas_acessorios                            66  1.924242    2.0  0.949666
papelaria                                  155  1.922581    2.0  0.922556
casa_construcao                             66  1.909091    2.0  0.836242
utilidades_domesticas                      586  1.889078    2.0  0.888473
cool_stuff                                 298  1.885906    2.0  0.906618
relogios_presentes                         589  1.881154    2.0  0.905088
pet_shop                                   147  1.857143    2.0  0.883641
moveis_escritorio                          261  1.854406    1.0  0.920843
cama_mesa_banho                           1221  1.850942    2.0  0.890235
esporte_lazer                              626  1.846645    2.0  0.899136
casa_conforto                               57  1.842105    2.0  0.902169
beleza_saude                               716  1.840782    1.0  0.908275
ferramentas_jardim                         318  1.817610    1.0  0.904635
telefonia                                  488  1.815574    2.0  0.875636
moveis_decoracao                           743  1.802153    1.0  0.888620
brinquedos                                 298  1.788591    1.0  0.890808
bebes                                      244  1.786885    1.0  0.877046
eletronicos                                248  1.778226    1.0  0.883502
construcao_ferramentas_construcao           71  1.774648    1.0  0.881511
automotivo                                 370  1.770270    1.0  0.870275
perfumaria                                 235  1.723404    1.0  0.879543
informatica_acessorios                     797  1.708908    1.0  0.867089
eletrodomesticos                            84  1.630952    1.0  0.875087
eletroportateis                             54  1.518519    1.0  0.770708

### (4.2) Worst product categories

👎 Store the five worst categories in terms of *average review score* into a variable called `worst_products`

In [ ]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

review_score                           
                              count      mean median       std
product_category_name                                         
informatica_acessorios          797  1.708908    1.0  0.867089
automotivo                      370  1.770270    1.0  0.870275
perfumaria                      235  1.723404    1.0  0.879543
eletrodomesticos                 84  1.630952    1.0  0.875087
eletroportateis                  54  1.518519    1.0  0.770708

👇 Create a `worst_products_review` DataFrame which contains the `worst_products` only.

In [ ]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id",
                        "review_score",
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
112,ff722b4c68783a4459a3adb9bb4e1d0d,3,informatica_acessorios,produt cheg pc consegu reconhec port usb,2,"[entreg, compr, receb, produt, ped]"
169,5f938e5f5f2e9a75710b54feeb9ea610,1,eletrodomesticos,medi pec serv,2,"[entreg, compr, receb, produt, ped]"
212,6b341682ab39af9fa00d72d7388c903b,3,automotivo,not compr produt corret cap intern ser outr,0,"[produt, receb, bom, receb produt, compr]"
253,b736ff4204044e49e39062584d06fa74,1,eletroportateis,loj anunc produt entreg outr,2,"[entreg, compr, receb, produt, ped]"
272,6f885e2fd69412264d9e74e805175d53,1,informatica_acessorios,receb bem rapid porem vei outr produt ped mode...,0,"[produt, receb, bom, receb produt, compr]"
...,...,...,...,...,...,...
98392,e9f0bde9a98ff79305964e9033ed8cd2,1,informatica_acessorios,produt cheg falt pess cond,1,"[produt, praz, gost, vei, entreg]"
98401,44380245bf1a49875542ea0e7f216986,1,informatica_acessorios,receb produt aind assim receb produt avali posi,0,"[produt, receb, bom, receb produt, compr]"
98512,970ae7c9df61910a0a30c38670db7ae5,3,informatica_acessorios,probl past multius pra notebook ped cor vermel...,2,"[entreg, compr, receb, produt, ped]"
98527,504403288b387de1c95a193909ba5dfd,3,perfumaria,cheg bem rapid produt razoa,1,"[produt, praz, gost, vei, entreg]"


### (4.3). Topics for the worst products

❓ What are the topics for the worst products ❓

In [ ]:
worst_products_reviews["most_important_topic"].value_counts()

2    628
0    527
1    385
Name: most_important_topic, dtype: int64

In [ ]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

[2, 0, 1]

In [ ]:
[topic_word_mixture[i] for i in bad_frequency]

[['entreg', 'compr', 'receb', 'produt', 'ped'],
 ['produt', 'receb', 'bom', 'receb produt', 'compr'],
 ['produt', 'praz', 'gost', 'vei', 'entreg']]

## (5) 🎁 Sellers...

* What kind of products were sold by the worst sellers ?
* What are the main reviews for the worst sellers ?

### (5.1) Worst Sellers

In [ ]:
from olist.seller import Seller
sellers = Seller().get_training_data()
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'n_orders', 'quantity', 'quantity_per_order', 'sales',
       'share_of_one_stars', 'share_of_five_stars', 'review_score'],
      dtype='object')

👇 Select the 10 worst sellers and store them into a variable called `worst_sellers`

In [ ]:
worst_sellers = sellers[["seller_id", "review_score", "profits"]].sort_values(
    by = "profits",
    ascending = True).head(10)
worst_sellers

,seller_id,review_score,profits
769,6560211a19b47992c3666cc44a7e94c0,3.937093,-21519.517
2358,4a3ca9315b744ce9f8e9374361493884,3.827873,-17752.708
1357,cc419e0650a3c5ba77189a1882b7556a,4.077586,-17581.158
945,ea8482cd71df3c1969d7b9473ff13abc,3.997371,-15542.248
315,8b321bb669392f5163d04c59e235e066,4.074547,-12886.431
453,1f50f920176fa81dab994f9023523100,4.130342,-9926.079
1213,d2374cbcbb3ca4ab1086534108cc3ab7,3.751880,-9017.608
1132,7c67e1448b00f6e969d365cea6b010ab,3.488798,-8017.611
2023,cca3071e3e9bb7d12640c9fbe2301306,3.846369,-7379.011
2687,1835b56ce799e6a4dc4eddc053f04066,3.637441,-7135.579


### (5.2) Products sold by the worst sellers

In [ ]:
products = Product().get_training_data() [["product_id", "category"]]
products

/Users/markbotterill/code/lewagon/olist/olist/order.py:32: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  orders.loc[:, 'order_delivered_customer_date'] = \
/Users/markbotterill/code/lewagon/olist/olist/order.py:34: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  orders.loc[:, 'order_estimated_delivery_date'] = \
/Users/markbotterill/code/lewagon/olist/olist/order.py:36: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df

,product_id,category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumery
...,...,...
31479,13b25797ad1e6d6c7cd3cbeb35f8ee7a,furniture_mattress_and_upholstery
31480,dc36a7859b743d8610a2bbbaea26ece9,furniture_mattress_and_upholstery
31481,107fde0930956120d1e13dd1062fbb46,furniture_mattress_and_upholstery
31482,726b4e18f00255e2e63491bcba3f60b8,furniture_mattress_and_upholstery


❓ What are the types of products sold by the worst sellers ❓

In [ ]:
sellers_product_category = data["order_items"].merge(products,
                                             on = "product_id", how = "left")[["seller_id", "category"]]

sellers_product_category

,seller_id,category
0,48436dade18ac8b2bce089ec2a041202,cool_stuff
1,dd7ddc04e1b6c2c614352b383efe2d36,pet_shop
2,5b51032eddd242adc84c38acab88f23d,furniture_decor
3,9d7a1d34a5052409006425275ba1c2b4,perfumery
4,df560393f3a51e74553ab94004ba5c87,garden_tools
...,...,...
112645,b8bc237ba3788b23da09c0f1f3a3288c,housewares
112646,f3c38ab652836d21de61fb8314b69182,computers_accessories
112647,c3cfdc648177fdbbbb35635a37472c53,sports_leisure
112648,2b3e4a2a3ea8e01938cabda2a3e5cc79,computers_accessories


In [ ]:
sellers_product_category.groupby("seller_id").count()

,category
seller_id,
0015a82c2db000af6aaaf3ae2ecb0532,3
001cca7ae9ae17fb1caed9dfb1094831,239
001e6ad469a905060d959994f1b41e4f,0
002100f778ceb8431b7a1020ff7ab48f,55
003554e2dce176b5555353e4f3555ac8,0
...,...
ffcfefa19b08742c5d315f2791395ee5,0
ffdd9f82b9a447f6f8d4b91554cc7dd3,20
ffeee66ac5d5a62fe688b9d26f83f534,14


### (5.3) Categories and topics for the worst sellers

🎁 Here are some useful functions:
- `focus_seller(seller_id)` to show the product categories sold by a seller
- `bad_reviews_seller` to show to topwords of the most frequent topic for one seller

In [ ]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [ ]:
bad_reviews_sellers = worst_products_reviews.merge(data["order_items"])
bad_reviews_sellers.head(3)

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic,most_important_words,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,ff722b4c68783a4459a3adb9bb4e1d0d,66,3,b8702f50a689621ab21ef200b55a5b8d,informatica_acessorios,"Produto chegou, mas meu PC não conseguiu reco...",produt cheg pc consegu reconhec port usb,2,"[entreg, compr, receb, produt, ped]",1,e76438822c17316d276c27e52e588931,5656537e588803a555b8eb41f07a944b,2018-01-10 22:07:28,24.89,17.63
1,5f938e5f5f2e9a75710b54feeb9ea610,17,1,d9ff0185a300435402f043124f530274,eletrodomesticos,Médio A peça não serviu,medi pec serv,2,"[entreg, compr, receb, produt, ped]",1,082175ee534338e4fd967076a6f848e5,d91fb3b7d041e83b64a00a3edfb37e4f,2018-05-10 16:54:16,52.00,18.24
2,6b341682ab39af9fa00d72d7388c903b,66,3,fd65651f855608ad1aa52ff4a741c31a,automotivo,Nota3 Comprar um produto correto na capa mas i...,not compr produt corret cap intern ser outr,0,"[produt, receb, bom, receb produt, compr]",1,1dce3fd38c13a3e74fe5000f33858442,72c5da29406b4234927b81855e7b64f6,2018-05-07 05:30:57,229.99,19.49


In [ ]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    if len(temp) > 0: # if seller appears in the bad reviews dataframe
        most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
        return topic_word_mixture[most_frequent_topic_seller]

❓For each of these worst sellers, show the most frequent product categories and words ❓

In [ ]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))


--------------------------------------------------
Focusing on the seller #6560211a19b47992c3666cc44a7e94c0...
seller_id                         category                 
6560211a19b47992c3666cc44a7e94c0  watches_gifts                1624
                                  fashion_bags_accessories      340
                                  audio                          32
                                  perfumery                      13
                                  computers_accessories          12
                                  sports_leisure                  7
                                  construction_tools_safety       1
dtype: int64
['entreg', 'compr', 'receb', 'produt', 'ped']
--------------------------------------------------
Focusing on the seller #4a3ca9315b744ce9f8e9374361493884...
seller_id                         category       
4a3ca9315b744ce9f8e9374361493884  bed_bath_table     1566
                                  home_confort        243
                 

🏁 Congratulations. You've learned some basics of NLP (Preprocessing + Vectorizing + NB/LDA) and we combined this new "expertise" with Decision Science

💾 Don't forget to `git add / commit / push`